In [ ]:
#!pip3 install lightly
#!pip install pytorch-lightning
#!pip install umap-learn
#!pip install umap
#!pip install umap-learn[plot]

In [108]:
import torch
import numpy as np
import torch.nn as nn
import torchvision
import lightly.models as models
import lightly.loss as loss
import lightly.data as data
import matplotlib.pyplot as plt
import pytorch_lightning as pl
from lightly.data import LightlyDataset
from tqdm import tqdm
#import umap.umap_ as umap
from torchvision.transforms import ToTensor
#import umap.plot

In [110]:
datasett = torchvision.datasets.STL10(root='data', split='unlabeled')
dataset2 = torchvision.datasets.STL10(root='data', split='test', transform=ToTensor())

In [83]:
#dataset = data.LightlyDataset(input_dir='data/stl10_png')
dataset = LightlyDataset.from_torch_dataset(datasett)
dataset2 = LightlyDataset.from_torch_dataset(dataset2)

In [84]:
collate_fn = data.ImageCollateFunction(input_size=96, cj_prob=0.5)

In [112]:
dataloader = torch.utils.data.DataLoader(
    dataset,                # pass the dataset to the dataloader
    batch_size=512,         # a large batch size helps with the learning
    shuffle=True,
    collate_fn=collate_fn
)
dataloader2 = torch.utils.data.DataLoader(
    dataset2,                # pass the dataset to the dataloader
    batch_size=128,         # a large batch size helps with the learning
    shuffle=False,

)

In [6]:
class SimCLR(nn.Module):
    def __init__(self, backbone):
        super().__init__()
        self.backbone = backbone
        self.projection_head = models.modules.SimCLRProjectionHead(
        input_dim=512,
        hidden_dim=512,
        output_dim=128)
        
    def forward(self, x):
        x = self.backbone(x).flatten(start_dim=1)
        z = self.projection_head(x)
        return z

In [72]:
resnet = torchvision.models.resnet18()
backbone = nn.Sequential(*list(resnet.children())[:-1])
device = ('cuda' if torch.cuda.is_available else 'cpu')
model = torch.load('models')
model.to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.08, weight_decay=1e-5)

In [100]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-5)
#5.55441

In [ ]:
criterion = loss.NTXentLoss(temperature=0.5)

In [101]:
print("Starting Training")
for epoch in range(10):
    total_loss = 0
    for (x0, x1), _, _ in tqdm(dataloader):
        x0 = x0.to(device)
        x1 = x1.to(device)
        z0 = model(x0)
        z1 = model(x1)
        loss = criterion(z0, z1)
        total_loss += loss.detach()
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    avg_loss = total_loss / len(dataloader)
    print(f"epoch: {epoch:>02}, loss: {avg_loss:.5f}")

Starting Training


100%|██████████| 196/196 [05:41<00:00,  1.74s/it]


epoch: 00, loss: 5.53476


100%|██████████| 196/196 [05:42<00:00,  1.75s/it]


epoch: 01, loss: 5.53385


...

epoch: 02, loss: 5.53335


100%|██████████| 196/196 [05:40<00:00,  1.74s/it]


epoch: 03, loss: 5.53428


  6%|▌         | 12/196 [00:20<05:08,  1.68s/it]


KeyboardInterrupt: 

In [102]:
torch.save(model, 'selfmodel')

In [129]:
model2 = SimCLR(backbone)
model2.to(device)

SimCLR(
  (backbone): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)

In [130]:
    preds = []
    label = []
    for (x0, x1) in tqdm(dataloader2):
        x0 = x0.to(device)
        z0 = model2(x0)
        preds.extend(z0.cpu().detach().numpy())
        label.extend(x1.cpu().detach().numpy())

100%|██████████| 63/63 [00:04<00:00, 15.11it/s]


In [131]:
len(label

SyntaxError: unexpected EOF while parsing (1157076781.py, line 1)

In [132]:
import umap.umap_ as umap
mapper = umap.UMAP().fit(preds)

In [133]:
import pandas as pd
hover_data = pd.DataFrame({'index':np.arange(8000),
                           'label':label})

hover_data['item'] = hover_data['label'].map(
    {
        0:'airplane',
        1:'bird',
        2:'car',
        3:'cat',
        4:'deer',
        5:'dog',
        6:'horse',
        7:'monkey',
        8:'ship',
        9:'truck',
    }
)

In [134]:
hover_data

,index,label,item
0,0,6,horse
1,1,7,monkey
2,2,5,dog
3,3,0,airplane
4,4,3,cat
...,...,...,...
7995,7995,9,truck
7996,7996,6,horse
7997,7997,8,ship
7998,7998,8,ship


In [135]:
import umap.plot
umap.plot.output_notebook()
p = umap.plot.interactive(mapper, 
                          labels=np.array(label), 
                          theme='fire', 
                          hover_data=hover_data, 
                          point_size=3)
umap.plot.show(p)

Loading BokehJS ...

In [104]:
import umap.umap_ as umap
mapper = umap.UMAP().fit(preds)
import pandas as pd
hover_data = pd.DataFrame({'index':np.arange(8000),
                           'label':label})

hover_data['item'] = hover_data['label'].map(
    {
        0:'airplane',
        1:'bird',
        2:'car',
        3:'cat',
        4:'deer',
        5:'dog',
        6:'horse',
        7:'monkey',
        8:'ship',
        9:'truck',
    }
)
import umap.plot
umap.plot.output_notebook()
p = umap.plot.interactive(mapper, 
                          labels=np.array(label), 
                          theme='fire', 
                          hover_data=hover_data, 
                          point_size=3)
umap.plot.show(p)

Loading BokehJS ...